In [1]:
pip install sgp4

     -------------------------------------- 160.8/160.8 kB 4.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pyproj

     ---------------------------------------- 6.1/6.1 MB 5.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
import pyproj
from time import process_time
from sgp4.api import Satrec
from sgp4.api import jday

In [18]:
t1_start = process_time()

In [19]:
TLE_file = "30000sats.txt"
TLEs = open(TLE_file, 'r')
L_Name = []
L_1 = []
L_2 = []
i = 1
for line in TLEs:
    j = i
    if i == 1:
        L_Name.append(line)
        j = 2
    elif i == 2:
        L_1.append(line[:69])
        j = 3
    elif i == 3:
        L_2.append(line[:69])
        j = 1
    i = j

In [20]:
dataframe = pd.DataFrame(columns = ['Satellite_name','Line_1','Line_2', 'Position_vector', 'Speed_vector'])
dataframe

,Satellite_name,Line_1,Line_2,Position_vector,Speed_vector


In [21]:
dataframe.Satellite_name = L_Name
dataframe.Line_1 = L_1
dataframe.Line_2 = L_2

In [22]:
jd, fr = jday(2022, 2, 4, 18, 5, 0)
L_PosVector = []
L_SpeedVector = []
for i in range(len(dataframe)):
    s = dataframe.Line_1[i]
    t = dataframe.Line_2[i]
    satellite = Satrec.twoline2rv(s, t)
    e, r, v = satellite.sgp4(jd, fr)
    L_PosVector.append(r)
    L_SpeedVector.append(v)

In [23]:
dataframe.Position_vector = L_PosVector
dataframe.Speed_vector = L_SpeedVector
dataframe.head(5)

,Satellite_name,Line_1,Line_2,Position_vector,Speed_vector
0,CALSPHERE 1 \n,1 00900U 64063C 23238.93868819 .00000734 0...,2 00900 90.1888 49.2514 0027605 147.5719 11...,"(3637.741063243929, 2816.586388140429, 5718.98...","(-4.5276089041592025, -3.5546511553377007, 4.6..."
1,CALSPHERE 2 \n,1 00902U 64063E 23238.53649546 .00000079 0...,2 00902 90.2022 52.6379 0017364 330.4562 95...,"(-6.033435361365596, -40.028652968072954, 7432...","(-5.524431321140139, -4.804024060083873, -0.04..."
2,LCS 1 \n,1 01361U 65034C 23238.84008827 .00000005 0...,2 01361 32.1414 91.5399 0006524 353.7994 6...,"(4367.657246705785, 6817.374579205074, 4275.48...","(-5.804639275536561, 2.6596021144071904, 1.679..."
3,TEMPSAT 1 \n,1 01512U 65065E 23238.89991498 .00000056 0...,2 01512 89.9355 215.5500 0067984 247.1913 175...,"(4415.761759101024, 3600.880085779401, -4983.4...","(-3.692425012365529, -2.997080468365593, -5.44..."
4,CALSPHERE 4A \n,1 01520U 65065H 23239.18937743 .00000117 0...,2 01520 89.9674 129.3289 0070572 113.9824 344...,"(-3148.764795171497, 3607.8618393429892, -5727...","(-3.7100606903408493, 4.241116201412246, 4.677..."


In [24]:
dataframe.drop(columns=['Line_1','Line_2','Speed_vector'])

,Satellite_name,Position_vector
0,CALSPHERE 1 \n,"(3637.741063243929, 2816.586388140429, 5718.98..."
1,CALSPHERE 2 \n,"(-6.033435361365596, -40.028652968072954, 7432..."
2,LCS 1 \n,"(4367.657246705785, 6817.374579205074, 4275.48..."
3,TEMPSAT 1 \n,"(4415.761759101024, 3600.880085779401, -4983.4..."
4,CALSPHERE 4A \n,"(-3148.764795171497, 3607.8618393429892, -5727..."
...,...,...
8445,2023-123A \n,"(1377.3024628542191, -2981.86037495711, 6170.3..."
8446,PROGRESS-MS 24 \n,"(-6135.323501112959, -2310.884665456729, -1979..."
8447,2023-126A \n,"(6559.496927759308, -1416.9466409044855, 2095...."
8448,JILIN-01 KUANFU 02A \n,"(444.17120502980225, 922.6998274557724, 6858.7..."


-----------------------------------PART-1 ENDS-------------------------------


In [25]:

def ecef2lla(pos_x, pos_y, pos_z):
  transformer = pyproj.Transformer.from_crs(
    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
    )
  lona, lata, alta = transformer.transform(pos_x,pos_y,pos_z, radians=False )
  f.write("%s,%s,%s\n" % (lata,lona,alta))
  return lona, lata, alta

In [26]:
f = open('file.txt', 'w')
for i in range(len(dataframe)):
  ecef2lla(dataframe['Position_vector'][i][0],dataframe['Position_vector'][i][1],dataframe['Position_vector'][i][2])

----------------PART-2 ENDS---------------------------------------------------

In [27]:
t1_end = process_time()
print(t1_end - t1_start)

16.0


In [15]:
37.94/28.21*100

134.4913151364764

In [16]:
pip install \
--extra-index-url=https://pypi.nvidia.com \
cudf-cu12==24.6.* \
dask-cudf-cu12==24.6.* \
cuml-cu12==24.6.* \
cugraph-cu12==24.6.*

SyntaxError: invalid syntax (3223064538.py, line 1)

In [28]:
conda install -c nvidia/label/cuda10.0 -c rapidsai/label/cuda10.0 -c numba -c conda-forge -c defaults cudf


Note: you may need to restart the kernel to use updated packages.


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: cudf


In [30]:
pip install cudf-cuda100

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for cudf-cuda100
Failed to build cudf-cuda100
  Running setup.py install for cudf-cuda100: started
  Running setup.py install for cudf-cuda100: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [34 lines of output]
  running bdist_wheel
  running build
  C:\Users\Varadaraju Harini\anaconda3\lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
    warnings.warn(
  installing to build\bdist.win-amd64\wheel
  running install
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\Varadaraju Harini\AppData\Local\Temp\pip-install-unh5wwzv\cudf-cuda100_3557845302ce4544ab35c3e273bf243d\setup.py", line 18, in <module>
      setup(name=pkg,
    File "C:\Users\Varadaraju Harini\anaconda3\lib\site-packages\setuptools\__init__.py", line 87, in setup
      return distutils.core.setup(**attrs)
    File "C:\Users\Varadaraju Harini\anaconda3\lib\site-packages\setuptools\